# StatsPipeline & Plotting Example

This example shows how to set up a statistical analysis pipeline using `StatsPipeline`. 

Additionally, it demonstrates the new plotting functions of `BioPsyKit`, that wrap the `boxplot()` function of `seaborn` and offer additional, often used features, such as adding significance brackets:
* `bp.plotting.feature_boxplot()` and `bp.plotting.multi_feature_boxplot()`
* as well as the derived functions specialized for saliva features: `bp.protocols.plotting.saliva_feature_boxplot()` and `bp.protocols.plotting.saliva_multi_feature_boxplot()`.

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.utils.dataframe_handling import multi_xs
from biopsykit.stats import StatsPipeline

import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
pg.options['round'] = 3

sns.set(style='ticks', context='talk')
plt.rcParams['figure.figsize'] = (10,5)
plt.close('all')

palette = bp.colors.fau_palette
sns.set_palette(palette)
palette

In [ ]:
data_path = Path("../example_data")

## Data Import

In [ ]:
sample_times = [-30, -1, 30, 40, 50, 60, 70]

condition_order = ["Control", "Intervention"]

### Raw Cortisol

In [ ]:
cort_samples = pd.read_csv(data_path.joinpath("cortisol_sample_stats.csv"))
cort_samples = cort_samples.set_index(["subject", "condition", "sample"])
cort_samples.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
# specialized function for plotting saliva data
bp.protocols.plotting.saliva_plot(
    data=cort_samples, 
    saliva_type="cortisol", 
    sample_times=sample_times, 
    test_times=[0, 30], 
    sample_times_absolute=True,
    ax=ax
)

### Cortsol Features

In [ ]:
auc = bp.saliva.auc(cort_samples, saliva_type="cortisol", sample_times=sample_times, compute_auc_post=True, remove_s0=True)
max_inc = bp.saliva.max_increase(cort_samples, saliva_type="cortisol", remove_s0=True)
slope = bp.saliva.slope(cort_samples, sample_idx=[1, 4], sample_times=sample_times, saliva_type="cortisol")

cort_features = pd.concat([auc, max_inc, slope], axis=1)
cort_features = bp.saliva.utils.saliva_feature_wide_to_long(cort_features, saliva_type="cortisol")
cort_features.head()

## Mixed ANOVA

In [ ]:
# construct some example to demonstrate analysis
data_example = multi_xs(cort_samples, [2, 3], level="sample")

In [ ]:
pipeline = StatsPipeline(
    steps=[
        ('prep', 'normality'),
        ('prep', 'equal_var'),
        ('test', 'mixed_anova'),
        ('posthoc', 'pairwise_ttests')
    ],
    params={
        'dv': 'cortisol',
        'between': "condition",
        'within': 'sample',
        'subject': 'subject',
        'padjust': 'bonf' # specify multicorrection method to be applied on the posthoc tests
    }
)

pipeline.apply(data_example);

### Display Results

In [ ]:
# display all results
pipeline.display_results()
# only significant results
# pipeline.display_results(sig_only=True)
# only significant results from the "posthoc" category (results from other categories will all be displayed)
# pipeline.display_results(sig_only="posthoc")

### Further functions of ``StatsPipeline``

In [ ]:
# analysis categories and their respective analysis steps
print(pipeline.category_steps)
# dictionary with analysis results per step
results = pipeline.results
# get results from normality check
display(results["normality"])
# return only results from one dategory
display(pipeline.results_cat("posthoc"))
# export the whole pipeline as Excel sheet
# pipeline.export_statistics()

## Repeated-Measurements ANOVA

In [ ]:
data_slice = data_example.xs("Control", level="condition")

pipeline = StatsPipeline(
    steps=[
        ('prep', 'normality'),
        ('prep', 'equal_var'),
        ('test', 'rm_anova'),
        ('posthoc', 'pairwise_ttests')
    ],
    params={
        'dv': 'cortisol',
        'within': 'sample',
        'subject': 'subject'
    }
)

pipeline.apply(data_slice)
pipeline.display_results()

## T-Tests

In [ ]:
cort_features.head()

`cort_features` contains multiple features that need to be analyzed individually. The analysis pipeline can be applied to each feature individually by specifying a column to group the dataframe by (`groupby` parameter). The result dataframes will then contain the `groupby` column as index level.

In [ ]:
pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"),
        ("prep", "equal_var"),
        ("test", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol",
        "groupby": "saliva_feature",
        "between": "condition"
    }
)

pipeline.apply(cort_features)
pipeline.display_results()

## Get Significance Brackets from `StatsPipeline`

`StatsPipeline.sig_brackets()` returns the significance brackets and the corresponding p-values to add to the plotting functions of `BioPsyKit`.

The method takes the following parameters (from the documentation):
* `stats_category_or_data`: either a string of the pipeline category to use for generating significance brackets or a dataframe with statistical if significance brackets should be generated from the dataframe
* `stats_type`: type of analysis performed ("between", "within", "interaction"). Needed to extract the correct information from the analysis dataframe
* `plot_type`: type of plot for which significance brackets are generated: "multi" if boxplots are grouped (by a ``hue`` variable), "single" (the default) otherwise
* `features`: feature(s) displayed in the boxplot. The resulting significance brackets will be filtered accordingly to only contain features present in the boxplot. It can have the following formats:
    * ``str``: only one feature is plotted in the boxplot  
      => returns significance brackets of only one feature
    * ``list``: multiple features are combined into *one* `Axes` object (i.e., no subplots)  
      => returns significance brackets of multiple features
    * ``dict``: if boxplots of features are organized in subplots then `features` needs to dictionary with the feature (or list of features) per subplot (``subplots`` is ``True``)  
      => returns dictionary with significance brackets per subplot
* `x`: name of column used as `x` axis in the boxplot. Only required if `plot_type` is "multi".
     

Data in a "single plot" (e.g., only to display `max_inc` feature) => filter by feature

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets("test", "between", "single", features="max_inc")
print(box_pairs)

Data in a "multi plot" (`x` variable is "saliva_feature", `hue` is the "between" variable)

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets("test", "between", "multi", x="saliva_feature")
print(box_pairs)

Data in a "multi plot" (`x` variable is "saliva_feature", `hue` is the "between" variable), but organized in subplots

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    "between", 
    "multi", 
    x="saliva_feature", 
    subplots=True
)
print(box_pairs)

Data in a "multi plot" (`x` variable is "saliva_feature", `hue` is the "between" variable), but organized in subplots. The features are now structured in a custom way (e.g., "max_inc" and "slope" should be placed into the same subplot).

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    "between", 
    "multi", 
    x="saliva_feature", 
    features={"auc": ["auc_i", "auc_g"], "inc": ["max_inc", "slope14"]}, 
    subplots=True
)
print(box_pairs)

## Plotting

### Plot Single Feature

This example shows how to plot a single feature in a boxplot using `bp.plotting.feature_boxplot`. The two conditions are plotted along the `x` axis.

In [ ]:
features = "max_inc"
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets("test", stats_type="between", features=features, plot_type="single")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=data_plot, 
    x="condition", 
    y="cortisol", 
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

### Plot Multiple Features

This example is the same as the one in the **Plot Single Feature** example above, but this time, the (single) feature is plotted along the `x` axis and the two groups are separated by the `hue` parameter. This makes it `plot_type` "multi" and thus requires to specify the `x` parameter in `StatsPipeline.sig_brackets()`.

In [ ]:
features = "max_inc"
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets("test", stats_type="between", plot_type="multi", features=features, x="saliva_feature")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=data_plot, 
    x="saliva_feature", 
    y="cortisol", 
    hue="condition",
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

This example shows how to use `bp.plotting.feature_boxplot` to plot actually multiple features along the `x` axis with the `hue` variable separating the conditions.

In this example, however, no feature has a statistically significant difference.

In [ ]:
features = ["auc_g", "auc_i"]
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets("test", stats_type="between", features=features, plot_type="multi", x="saliva_feature")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=data_plot, 
    x="saliva_feature", 
    y="cortisol", 
    hue="condition",
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

### Plot Multiple Features in Subplots

This example shows how to use `bp.plotting.multi_feature_boxplot()` to plot multiple features as boxplots, but into single subplots. The function allows to group certain features together into the same subplot.

In [ ]:
features = {"auc": ["auc_g", "auc_i"], "max_inc": "max_inc", "slope14": "slope14"}

box_pairs, pvalues = pipeline.sig_brackets("test", stats_type="between", features=features, plot_type="multi", x="saliva_feature", subplots=True)

data_plot = cort_features.copy()

bp.plotting.multi_feature_boxplot(
    data=data_plot, 
    x="saliva_feature", 
    y="cortisol", 
    hue="condition",
    group="saliva_feature",
    features=features,
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }
)
fig.tight_layout()

### Use Specialized `saliva_feature_plot` functions

In addition to the "general-purpose" plotting functions `BioPsyKit` also offers specialized plotting functions for saliva features since plotting saliva data is a commonly performed task. These functions already offer a better styling of axis and labels.

In [ ]:
# rerun the pipeline
pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"),
        ("prep", "equal_var"),
        ("test", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol",
        "groupby": "saliva_feature",
        "between": "condition"
    }
)

pipeline.apply(cort_features)
pipeline.display_results()

#### Plot Single Feature

In [ ]:
features = "max_inc"
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets("test", stats_type="between", features=features, plot_type="single")

fig, ax = plt.subplots()
bp.protocols.plotting.saliva_feature_boxplot(
    data=data_plot, 
    x="condition", 
    saliva_type="cortisol", 
    feature=features,
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

#### Plot Multiple Features

In [ ]:
features = ["auc_g", "auc_i"]
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets("test", stats_type="between", features=features, plot_type="multi", x="saliva_feature")

fig, ax = plt.subplots()
bp.protocols.plotting.saliva_feature_boxplot(
    data=data_plot, 
    x="saliva_feature",
    saliva_type="cortisol", 
    hue="condition",
    feature=features,
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

#### Plot Multiple Features in Subplots

In [ ]:
features = {"auc": ["auc_g", "auc_i"], "max_inc": "max_inc", "slope14": "slope14"}

box_pairs, pvalues = pipeline.sig_brackets("test", stats_type="between", features=features, plot_type="multi", x="saliva_feature", subplots=True)

data_plot = cort_features.copy()

bp.protocols.plotting.saliva_multi_feature_boxplot(
    data=data_plot, 
    saliva_type="cortisol", 
    features=features,
    hue="condition",
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }
)
fig.tight_layout()